# SIXT33N version C
## Phase 3: Controls

### EE 16B: Designing Information Devices and Systems II, Fall 2015

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-

##Table of Contents

* [Introduction](#intro)
* [Part 1: The SIXT33N Car](#part1)
* [Part 2: Sensors and Actuators](#part2)
* [Part 3: Open Loop System Modeling](#part3)
* [Part 4: Closed Loop Simulation and Implementation](#part4)
* [Part 5: Implementing Turns](#part5)

<a id='intro'></a>
## Introduction

In this phase we will first build the SIXT33N chassis and the sensors (wheel encoders) and actuators (motor drivers). We will also power the car up using a 9V battery. Next, we will model the the car as an open loop system. We will gather some dynamics data to figure out the variables in our model and lastly build a closed loop controller. By the end of the phase, you should be able to control the trajectory of the car, which will be integrated with the other components of the project in the last phase.

As a reminder, there are five different commands you will implement, at minimum. (If you want to implement more commands then go for it!)

- Straight vertical line from bottom to top = Fast
- Straight vertical line from top to bottom = Slow
- Straight horizontal line from right to left = Turn Left
- Straight horizontal line from left to right = Turn Right
- Clockwise circle starting at 12 'o clock = Party Mode

<b>Remember to document all design choices you made and explain them in the project report.</b>

The goals of this phase are as follows:
- Motor driver circuit
- Wheel encoder circuit
- System modelling
- Controller design
- Closed loop simulation
- Move at constant speed + direction

<a id='part1'></a>
## <span style="color:blue">Part 1: The SIXT33N Car</span>

###Materials
- Chassis kit
- 2 Photointerrupters
- Soldering iron + solder
- Small pieces of cardboard

Build the SIXT33N car from the kit. The instructions can be found in the <a href="http://inst.eecs.berkeley.edu/~ee16b/fa15/proj/">project page</a>.

<a id='part2'></a>
## <span style="color:blue">Part 2: Sensors and Actuators</span>

###Materials
- Part 1 setup
- 4 BS250 PMOS transistors
- 2 1k$\Omega$ resistors
- 2 300$\Omega$ resistors
- Switch
- 9V battery + battery leads

## Motor Drivers
To be able to move, the car needs motor drivers. The DC motors we use are usually driven by a Pulse Width Modulated (PWM) signal which will be produced by the Launchpad. A PWM signal is simply a square wave at a certain frequency but with variable duty cycle (percentage time the signal is high in a period). However, the wheels need a higher voltage and current source than the Launchpad can supply. To solve this issue, we will hook the wheels up to 5V and use a PMOS transistor control whether the motor is powered or not. Recall that a PMOS conducts when the gate-source voltage is higher than a threshold. In the circuit below, the terminal connected to the 5V power source is the source terminal. That means when the Launchpad outputs a 0 the motor is driven, and when it outputs a 1 the motor is off. Keep this in mind - it will be handy later.

<center>
<img width='220px' src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/proj-motor-driver.png">
</center>

<b>Construct the circuit above for both wheels using the <a href="http://www.vishay.com/docs/70209/70209.pdf">BS250</a> PMOS on the breadboard</b>. Note that the two PMOSes for each driver are connected in parallel. The breadboard should go on the car chassis. The pin diagram of the PMOS is reproduced below. Be careful when you connect the PMOS and double check which terminal is which since it is easy to damage if you hooked it up incorrectly. Note that the diagram says top view. Also note that the diagram below is not in the same orientation as the one in the circuit above. The motor is reversible so connect it any way for now. If it turns out to go backwards later, just reverse the two terminals of the motor. Leave the gate of the PMOS connected to 5V for now - we will connect it to the Launchpad later.

<center>
<img width='140px' src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/proj-bs250.png">
</center>

## Wheel Encoders

Besides driving, the car also needs to be able to sense how far it has travelled and the speed at which it is travelling. To do this, we will use a <a href="http://www.sharp-world.com/products/device/lineup/data/pdf/datasheet/gp1a58hr_e.pdf">photointerrupter</a> and a wheel encoder. When you built the car you should have attached a disk with holes to the motors. You should also have soldered wires to the photointerrupters. We will now connect these in the correct way. The diagram below shows the photointerrupter from the top (i.e. black plastic facing you, pins going into the screen).

<center>
<img width='350px' src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/proj-photoint-top.png">
</center>

The right side of the component is an infrared LED. The left side of the component is a photodiode which conducts current only when it sees the light from the infrared LED, just like the solar cells you used last semester. There is some circuitry to make the signal a digital signal so the output is a square wave. If the space between the LED and the photodiode is empty, it will output a high signal. If the space is blocked and the light is obscured it outputs a low signal. From this, we can imply how fast the wheel is moving from the holes in the encoder, because the square wave will be at higher frequencies for high speeds and low frequencies for low speeds.

Connect the 5 terminals of the photointerrupter as shown below. The diagram is drawn to mimic the diagram above, but <b>note that these views are with the pins away from you</b>.

<center>
<img width='450px' src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/proj-photoint-circuit.png">
</center>

Add the resistors on the breadboard and connect the components correctly for both photointerrupters. Again, leave out the Launchpad pin - we will connect this later.

## Battery

Rather than trying to run after your car while carrying a power soure, we will use a 9V battery as the input to your voltage regulator. However, to make it easier and safer to turn the battery on and off, we will add a switch. Use the middle pin of the switch and one of the other pins to connect the 9V battery lead to the voltage regulator input. The ground of the 9V battery lead should go to the ground of the circuit.

Note that if you are testing things at the bench, use the benchtop power supply. The battery does not contain a lot of energy and the motors take a lot of power. If you leave it running it will only last a little over an hour. If the voltage of the battery drops below 7V, ask for a new one from your GSI. <b>Do not connect both the power source and the battery at the same time. Don't do it.</b>

You will power the Launchpad through its 5V pin. Connect the 5V output of the voltage regulator to a separate row in the breadboard and add a large capacitor from that row to ground. Use this row (the hole next to the capacitor leg) to power up the Launchpad's 5V pin. This will ensure a stable power source for the Launchpad.

<a id='part3'></a>
## <span style="color:blue">Part 3: Open Loop System Modeling</span>

###Materials
- Part 2 setup
- Launchpad + USB cable

Before trying to control the SIXT33N, we will first characterize the open loop reponse of the system. To make modeling easier, we will separate the two wheels' model. The model that we create for both wheels can be identical because they are symmetric. We will use the open loop model you have seen many times in lecture and homework where the state is defined as the distance the wheel has travelled since reset and the velocity of the wheel. The open loop model has an input $u[k]$ which is the equivalent of force (in reality, the change in PWM level) and outputs both the distance and the velocity (thus the $C$ matrix is just the identity matrix).

$$\begin{bmatrix} d[k+1] \\ v[k+1] \end{bmatrix} = A \begin{bmatrix} d[k] \\ v[k] \end{bmatrix} + B u[k]$$

$$A = \begin{bmatrix} 1 & T_s \\ 0 & 1 \end{bmatrix} \qquad B = \begin{bmatrix} b_1 \\ b_2 \end{bmatrix}$$

All distances are measured in cm (each encoder tick corresponds to 1cm of wheel travel) while time is measured in ms. As you can see, there are 2 unknown variables, namely $b_1$ and $b_2$. To determine these parameters, we will perform least squares regression on each row. You will do this for both wheels separately.

<b>Your car wheels are going to move in this step. Make sure that the car is suspended above the desk, either with your hand or the clamp at the desk. Don't let your car jump off the bench.</b>

First, you will connect the car to your Launchpad. Remove 5V from the gate of the MOSFET. <b>Do not forget to remove the 5V</b>. Now, connect the Launchpad pins that control the motors and encoders:

 - LEFT_MOTOR P2_0
 - LEFT_ENCODER P2_4
 - RIGHT_MOTOR P1_5
 - RIGHT_ENCODER P1_4
 
Now, you will collect the data by uploading the sketch <b>`dynamics_data.ino`</b>. Glance through the sketch - it records the position and the average time between encoder ticks (i.e. 1/velocity) of each wheel while slightly varying the input signal $u[k]$. The sampling period is $T_s = 200$ms. Power the Launchpad from the voltage regulator using the 5V pin. Place the car in a large empty space and hit the reset button. After a couple seconds it will start moving and a couple seconds after that it starts collecting data. When it stops moving bring the car to your PC and connect the Launchpad to your PC using a USB cable. Open Serial Monitor in Energia - you will see a lot of data printed. Copy the data between two "Start" lines (150 lines total) to a text file called <b>`data.txt`</b>. This data will be used in the cells below.

Now that we have some data, we can try performing least square regression. Formulate the least squares problem for each row of the matrix equation above on a piece of paper and code it up in the cells below. Unfortunately, `np.linalg.lstsq` does not work if you only have 1 column in the `A` matrix, so you will have to compute the solution manually. The functions `.T` and `np.linalg.inv` will be helpful here. Execute the same code for both the left and right wheel data and record the values of $b_1$ and $b_2$ for each wheel.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
DATA_LEN = 150
data = np.zeros((DATA_LEN,5))
f = open('data.txt', 'r')
for i,line in enumerate(f.readlines()):
    data[i,:] = str.split(line, ',')
f.close()

In [ ]:
# Load the data
Ts = 200
u_k = np.matrix(data[1:,0] - data[:DATA_LEN-1,0]).T

d_k_left = np.matrix(data[:DATA_LEN-1,1]).T     # d[k]
d_k1_left = np.matrix(data[1:,1]).T             # d[k+1]
v_k_left = np.matrix(1/data[:DATA_LEN-1,2]).T   # v[k]
v_k1_left = np.matrix(1/data[1:,2]).T           # v[k+1]

d_k_right = np.matrix(data[:DATA_LEN-1,3]).T
d_k1_right = np.matrix(data[1:,3]).T
v_k_right = np.matrix(1/data[:DATA_LEN-1,4]).T
v_k1_right = np.matrix(1/data[1:,4]).T

In [ ]:
# Use this block to find the left wheel parameters
d_k = d_k_left
d_k1 = d_k1_left
v_k = v_k_left
v_k1 = v_k1_left

In [ ]:
# Use this block to find the right wheel parameters
d_k = d_k_right
d_k1 = d_k1_right
v_k = v_k_right
v_k1 = v_k1_right

In [ ]:
# YOUR CODE HERE #
A = 
b = 
b1 = 
print('b1 = ' + str(b1))

In [ ]:
# YOUR CODE HERE #
A = 
b = 
b2 = 
print('b2 = ' + str(b2))

To see how good the model is, try simulating the open loop model. The cells below takes the initial distance and velocity and plots the distance and velocity the model predicts. The result should be pretty bad compared to the actual data, but it's good enough to model the system - the feedback loop will (magically) take care of it!

In [ ]:
# Simulate velocity and distance with open loop model
sim = np.zeros((2, DATA_LEN))
sim[0,0] = d_k1[0]
sim[1,0] = v_k1[0]

# Open loop update matrices
A = np.matrix([[1, Ts], [0, 1]])
B = np.matrix([[b1], [b2]])

for i in range(1, DATA_LEN):
    sim[:,i] = (np.dot(A, np.matrix(sim[:,i-1]).T) + np.dot(B, np.matrix(u_k[i-1]).T)).T

In [ ]:
plt.plot(d_k1, 'black')
plt.plot(d_k + Ts*v_k + b1*u_k, 'red')
plt.plot(sim[0,:], 'blue')
plt.xlabel('Time (samples)')
plt.ylabel('distance (cm)')
plt.legend(['collected data', 'least squares fit', 'simulated'], loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
plt.plot(v_k1, 'black')
plt.plot(v_k + b2*u_k, 'red')
plt.plot(sim[1,:], 'blue')
plt.xlabel('Time (samples)')
plt.ylabel('velocity (cm/ms)')
plt.legend(['collected data', 'least squares fit', 'simulated'], loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
b1_left = 
b2_left = 
b1_right = 
b2_right = 

Before we move on, find the eigenvalues of the matrix `A`.

In [ ]:
print('A:')
print(A)
e = np.linalg.eig(A)
print('eigenvalues:')
print(e[0])
print('eigenvectors:')
print(e[1])

<span style='color:red'>**What do the two eigenvalues tell you?**</span>

YOUR ANSWER HERE

<a id='part4'></a>
## <span style="color:blue">Part 4: Closed Loop Simulation and Modeling</span>

###Materials
- Part 3 setup

Now that we have an open loop model, we can build a closed loop controller for each wheel. To simplify things, we will decouple the controller of the two wheels. However, your car will have a tough time going stright if the wheels have no idea what the other is doing, so the wheels will influence each other through the inputs. The left wheel controller has 2 inputs, namely the right wheel distance and the desired velocity, while the right wheel controller has 2 inputs, namely the left wheel distance and the desired velocity.

<span style='color:red'>**Using the information above, draw a block diagram of both wheels' controllers.**</span> Show this diagram to your GSI before moving on. Keep this somewhere, as you will use it in your report.

<span style='color:red'>**Now derive the closed loop update matrix A_CL from your models.**</span> Keep this somewhere, as you will use it in your report.

Once you have derived the closed loop, you will want to choose the controller gains $\alpha_1$ and $\alpha_2$ such that your system is stable. Use the gains to move the eigenvalues of the update matrix to be inside the unit circle. To do this, first write out the equation you would solve to find the eigenvalues. Group the $\lambda_{CL}^2$ terms together and the $\lambda_{CL}$ terms together. If our desired eigenvalues are $\lambda_{d1}$ and $\lambda_{d2}$, this equation should look something like (gain terms not shown):
$$(\lambda_{CL} - \lambda_{d1})(\lambda_{CL} - \lambda_{d2}) = \lambda_{CL}^2 - (\lambda_{d1} + \lambda_{d2}) \lambda_{CL} + \lambda_{d1} \lambda_{d2}$$

Equating the terms from the equation you derived and the equation above, you should get a system of 2 equations with 2 unknowns. Thus, given $\lambda_{d1}$ and $\lambda_{d2}$, you can solve for the controler gain values $\alpha_1$ and $\alpha_2$.

Now is the time for you to experiment! Using an equation solver such as Wolfram Alpha, solve for the controller gain values given some desired eigenvalues. Remember that $\lambda_{d1}$ and $\lambda_{d2}$ have to be complex conjugates and should be inside the unit circle. Try different values in the cells below to see how your system behaves. (You might also want to try a value outside the unit circle to see how it behaves). Remember to document your experiment results to discuss in the project report. Finally choose a controller value that gives a reasonable fast convergence time.

In [ ]:
SIM_LENGTH = 100

# YOUR CODE HERE #
alpha1 = 
alpha2 = 

A = np.matrix([[1, Ts], [0, 1]])
B = np.matrix([[b1], [b2]])
alpha = np.matrix([alpha1, alpha2])

x = np.zeros((2,SIM_LENGTH))
x[1,0] = 0.0004
const_vel = 0.05
vd = np.ones((SIM_LENGTH,1))*const_vel
yow = np.linspace(0,SIM_LENGTH*const_vel*Ts, SIM_LENGTH)

for i in range(0,SIM_LENGTH-1):
    xd = [yow[i], vd[i]]
    u = np.dot(alpha, np.subtract(xd, x[:,i]))[0,0]
    x[:,i+1] = np.add(np.dot(A, np.matrix(x[:,i]).T), np.dot(B,u)).T

In [ ]:
print('A_CL:')
# YOUR CODE HERE #
A_CL = 
print(A_CL)
e = np.linalg.eig(A_CL)
print('eigenvalues:')
print(e[0])
print('eigenvectors:')
print(e[1])

In [ ]:
plt.plot(x[0,:].T)
plt.plot(yow.T)
plt.xlim([0,SIM_LENGTH-2])
plt.legend(['simulation', 'desired'])

In [ ]:
plt.plot(x[1,:].T)
plt.plot(vd)
plt.xlim([0,SIM_LENGTH-2])
plt.legend(['simulation', 'desired'])

Now implement the controller in the Launchpad. The sketch <b>`closed_loop.ino`</b> contains some code that implements the closed loop control. Fill in the parts labeled `YOUR CODE HERE`:
- Add the controller gain `alpha1` and `alpha2` in lines 39-40
- Add the new $u[k]$ value in lines 99-100. The functions `input_left` and `input_right` updates the left and right controllers. The variables of interest are: `alpha1_left`, `alpha2_left`, `alpha1_right`, `alpha2_right`, `left_position`, `right_position`, `left_target_speed`, `right_target_speed`, `left_current_speed` and `right_current_speed`

Upload the sketch to your Launchpad. Your car wheels are going to move in this step, too. Make sure that the car is suspended above the desk.
 
Check that the wheels are moving as you expect them to. If you use your hand to put some friction on the wheels, does the feedback correct for it?

Once you are satisfied that your motors work and your closed loop is implemented, put the car on the ground. Click the reset button, the SIXT33N should go in a straight line. Does it behave how you expect? If it behaves erroneously (for example the controller oscillates), go back to the simulation and try different eigenvalues. This is an artifact of using a linear model as an approximation of the system and can be solved by picking a more suitable controller gain.

<span style='color:red'>**What are some reasons why the simulation might behave differently from the car? What about the differences between when the car is on the ground versus when it is on the desk?**</span>

<a id='part5'></a>
## <span style="color:blue">Part 5: Implementing Turns</span>

###Materials
- Part 4 setup

Now go back to your closed loop control and think of a way to implement a turn given the system that you already have. This should be a simple addition to your model - think about the easiest signal you can manipulate to get the car to turn.

<span style='color:red'>**Show your GSI the new block diagram.**</span>

Once your GSI agrees, implement the turning on the Launchpad. The sketch has a variable `count` which just increments every time the timestep $k$ is incremented. Write some code such that the SIXT33N moves straight for the first 5 seconds, then turns right for the second 5 seconds, then turns left for the third 5 seconds.

<span style='color:red'>**Show your GSI the controlled SIXT33N!**</span>

In the next phase, you will integrate all of your parts and use the results of the PCA to control the SIXT33N through the microphone.